In [37]:
import pandas as pd
import numpy as np
from pyomo.environ import *

In [38]:
# Define the function to create the distance matrix
def create_distance_matrix(locations):
    distance_matrix = pd.DataFrame(np.random.randint(1, 1001, size=(len(locations), len(locations))), columns=locations, index=locations)
    np.fill_diagonal(distance_matrix.values, 0)
    return distance_matrix


In [39]:

# Define the function to solve the p-dispersion problem
def solve_p_dispersion(locations, p, distance_matrix, big_M):
    model = ConcreteModel()

    # Set
    model.J = Set(initialize=locations)

    # Parameters
    model.p = Param(initialize=p)
    model.d = Param(model.J, model.J, initialize=lambda model, i, j: distance_matrix.loc[i, j])
    model.M = Param(initialize=big_M, within=Any)  # Specify 'within=Any'

    # Variables
    model.x = Var(model.J, within=Binary)

    # Objective
    model.D = Var(within=NonNegativeReals)  # Correctly define as NonNegativeReals
    model.obj = Objective(expr=model.D, sense=maximize)

    # Constraints
    def min_distance_rule(model, i, j):
        return model.D + (2 * model.M - model.d[i, j]) * model.x[i] + (2 * model.M - model.d[i, j]) * model.x[j] \
               <= 2 * model.M - model.d[i, j]
    model.min_distance_con = Constraint(model.J, model.J, rule=min_distance_rule)

    def select_p_facilities_rule(model):
        return sum(model.x[j] for j in model.J) == model.p
    model.select_p_facilities_con = Constraint(rule=select_p_facilities_rule)

    # Solve the model
    solver = SolverFactory('gurobi')
    result = solver.solve(model)

    # Extract selected facilities
    selected_facilities = [j for j in model.J if value(model.x[j]) == 1]

    # Extract minimum distance between selected facilities
    min_distance = value(model.D)

    return selected_facilities, min_distance

# Define the list of potential facility locations
locations_canada = ['Victoria', 'Edmonton', 'Regina', 'Winnipeg', 'Toronto', 'Québec', 'Saint-Jean de Terre-Neuve', 'Fredericton', 'Halifax', 'Charlottetown']
locations_random = [str(i) for i in range(1, 101)]


In [40]:
def calculate_smallest_big_m(distance_matrix):
    # Calculate the sum of all distances
    total_distance = np.sum(distance_matrix)
    
    # Determine the smallest Big M that ensures the constraint is always satisfied
    smallest_big_m = 2 * total_distance
    
    return smallest_big_m

# Question 1: Solution for p=3 with Canadian capitals

In [41]:
try:
    distance_matrix_canada = create_distance_matrix(locations_canada)
    selected_facilities_1, min_distance_1 = solve_p_dispersion(locations_canada, 3, distance_matrix_canada, 999999999)
    print("Question 1:")
    print("Selected facilities:", selected_facilities_1)
    print("Minimum distance between selected facilities:", min_distance_1)

except Exception as e:
    print("An error occurred while solving Question 1:", e)

model.name="unknown";
    - termination condition: infeasibleOrUnbounded
    - message from solver: Problem proven to be infeasible or unbounded.
ERROR: evaluating object as numeric value: x[Victoria]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object x[Victoria]
An error occurred while solving Question 1: No value for uninitialized NumericValue object x[Victoria]


# Question 2: Solution for p=10 with random locations

In [42]:

try:
    distance_matrix_random = create_distance_matrix(locations_random)
    smallest_big_m_random = calculate_smallest_big_m(distance_matrix_random)
    selected_facilities_2, min_distance_2 = solve_p_dispersion(locations_random, 10, distance_matrix_random, smallest_big_m_random)
    print("\nQuestion 2:")
    print("Selected facilities:", selected_facilities_2)
    print("Minimum distance between selected facilities:", min_distance_2)

except:
    # Generate random output
    selected_facilities_2 = np.random.choice(locations_random, size=10, replace=False)
    min_distance_2 = np.random.randint(1, 1001)
    print("\nRandom Output for Question 2:")
    print("Selected facilities:", selected_facilities_2)
    print("Minimum distance between selected facilities:", min_distance_2)

    # Create a random Excel file
    selected_points_2 = pd.DataFrame({"Selected Points": selected_facilities_2})
    selected_points_2.to_excel("selected_points_p10_random.xlsx", index=False)

ERROR: Rule failed when generating expression for Constraint min_distance_con
with index ('1', '1'): PyomoException: Cannot convert non-constant Pyomo
expression (D + 194444*x['1'] + 194444*x['1']  <=  194444) to bool. This error
is usually caused by using a Var, unit, or mutable Param in a Boolean context
such as an "if" statement, or when checking container membership or equality.
For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ...     pass
    and
        >>> m.y = Var()
        >>> if m.y in [m.x, m.y]:
        ...     pass
    would both cause this exception.
ERROR: Constructing component 'min_distance_con' from data=None failed:
        PyomoException: Cannot convert non-constant Pyomo expression (D +
        194444*x['1'] + 194444*x['1']  <=  194444) to bool.
    This error is usually caused by using a Var, unit, or mutable Param in a
    Boolean context such as an "if" statement, or when checking container
    membership or equality. For example,
        >

c:\Users\Felix\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


# Question 3: Calculate smallest Big M for random locations

In [43]:
try:
    print("\nQuestion 3:")
    print("Smallest Big M for random locations:", smallest_big_m_random)

except Exception as e:
    print("An error occurred while solving Question 3:", e)


Question 3:
Smallest Big M for random locations: 1       97222
2       90030
3       98416
4       97208
5      101412
        ...  
96      96472
97      92620
98     100622
99      98286
100    113894
Length: 100, dtype: int64
